# MODELE XGBOOST TARGET CALIBRE + SOUMISSION

## PREPROCESSING

In [ ]:
import pandas as pd
import numpy as np
import panel as pn

data = pd.read_csv('result_requetes.csv')
#data.head()

In [ ]:
#On affecte le bon type aux variables qualitatives
data["city"]=pd.Categorical(data["city"],ordered=False)
data["language"]=pd.Categorical(data["language"],ordered=False)
data["group"]=pd.Categorical(data["group"],ordered=False)
data["brand"]=pd.Categorical(data["brand"],ordered=False)

data["mobile"]=pd.Categorical(data["mobile"],ordered=False)
data["parking"]=pd.Categorical(data["parking"],ordered=False)
data["pool"]=pd.Categorical(data["pool"],ordered=False)
data["children_policy"]=pd.Categorical(data["children_policy"],ordered=False)

data["hotel_id"]=pd.Categorical(data["hotel_id"],ordered=False)
data["avatar_id"]=pd.Categorical(data["avatar_id"],ordered=False)

data["price"]=pd.DataFrame(data["price"], dtype=float)

#data.dtypes
#data.head()

## APPRENTISSAGE DE MODELE

Avec TargetEncoder / MEstimateEncoder

In [ ]:
#!pip install category_encoders > /dev/null 2>&1   EN LOCAL ?!

In [ ]:
from category_encoders import TargetEncoder
#from category_encoders import MEstimateEncoder

In [ ]:
X = data[["nb_requete","date","stock","city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]

In [ ]:
Y = data[["price"]]

NE PAS EXECUTER LES 2 CELLULES SUIVANTES

In [ ]:
#X=data[["city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]
#enc = TargetEncoder()
#Xenc = enc.fit_transform(X,Y)
#dataDum=pd.DataFrame(Xenc)
#dataDum

In [ ]:
#dataQuant = data[["nb_requete","date","stock"]]
#dfC = pd.concat([dataQuant,dataDum],axis=1)
#dfC

In [ ]:
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(dfC, Y, test_size=0.1, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [ ]:
y_train = np.ravel(y_train)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

#n_estimators + grand? 100 par défaut
#max_features?
#max_depth? 3 par défaut
xgb = make_pipeline(
    ('Target',TargetEncoder()),
    ('Scale',StandardScaler()),
    #('XGB',GradientBoostingRegressor(n_estimators=300,max_depth=10)),
    ('GridSearch',GridSearchCV(GradientBoostingRegressor(),[{"n_estimators":[100,300,500],"max_features":[1.,0.5,0.2],"max_depth":[3,7,10]}],cv=5,n_jobs=-1)),
    #('GridSearch',GridSearchCV(GradientBoostingRegressor(),[{"max_features":[1.,0.5,0.2]}],cv=5,n_jobs=-1)),
)

# RandomizedSearchCV ???

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
# POUR ENREGISTRER/CHARGER LE MODELE
from joblib import dump, load
dump(xgb, 'XGB_model_saved.joblib')
#xgb_loaded = load('XGB_model_saved.joblib')

In [ ]:
xgb.named_steps['GridSearch'].best_score_

In [ ]:
xgb.named_steps['GridSearch'].best_params_

In [ ]:
print(f"XGB score: {xgb.score(X_test, y_test):.2f}")

## TEST DU MODELE SUR LE FICHIER DEFI

In [ ]:
test = pd.read_csv('./defi-ia-2023/test_set3.csv')
test.head()

In [ ]:
#On affecte le bon type aux variables qualitatives
test["city"]=pd.Categorical(test["city"],ordered=False)
test["language"]=pd.Categorical(test["language"],ordered=False)
test["group"]=pd.Categorical(test["group"],ordered=False)
test["brand"]=pd.Categorical(test["brand"],ordered=False)

test["mobile"]=pd.Categorical(test["mobile"],ordered=False)
test["parking"]=pd.Categorical(test["parking"],ordered=False)
test["pool"]=pd.Categorical(test["pool"],ordered=False)
test["children_policy"]=pd.Categorical(test["children_policy"],ordered=False)

test["hotel_id"]=pd.Categorical(test["hotel_id"],ordered=False)
test["avatar_id"]=pd.Categorical(test["avatar_id"],ordered=False)

#test.dtypes
#test.head()

NE PAS EXECUTER LES 3 CELLULES SUIVANTES

In [ ]:
#T=test[["city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]
#Tenc=enc.transform(T)

In [ ]:
#testDum=pd.DataFrame(Tenc)
#testDum

In [ ]:
#testQuant = test[["nb_requete","date","stock"]]
#Xtest = pd.concat([testQuant,testDum],axis=1)
#Xtest.head()

In [ ]:
T = test[["nb_requete","date","stock","city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]

In [ ]:
#Yprev=xgb.predict(Xtest)
Yprev=xgb.predict(T)

In [ ]:
Yprev0=np.around(Yprev, decimals=1)

In [ ]:
Yindice = pd.DataFrame(np.arange(0,len(Yprev0)),columns = ['index'])

In [ ]:
Yprice=pd.DataFrame(Yprev0,columns = ['price'])
Ysub=Yindice.join(Yprice, on=None, how='right', lsuffix='', rsuffix='', sort=False)
Ysub.to_csv('./submission/sample_submissionXGB.csv', index= False)